In [ ]:
# Updated 05.10.2020

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
import biogeme.results as res
from biogeme.models import logit

pandas = pd.read_table("netherlandsRP_w.dat")
database = db.Database("netherlands",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)


In [ ]:
#WEIGHT NORMALIZATION
sumWeights = database.data['Weights'].sum()
S= database.getSampleSize()
sampleNormalizedWeight = Weights * S / sumWeights

In [ ]:
#MODEL SPECIFICATION (from estimation file)
ASC_CAR = Beta('ASC_CAR',0,-10,10,0)
ASC_RAIL = Beta('ASC_RAIL',0,-10,10,1)
BETA_TIME_RAIL = Beta('BETA_TIME_RAIL',0,-10,10,0)
BETA_TIME_CAR = Beta('BETA_TIME_CAR',0,-10,10,0)
BETA_COST_AGE1 = Beta('BETA_COST_AGE1',0,-10,10,0)
BETA_COST_AGE2 = Beta('BETA_COST_AGE2',0,-10,10,0)
BETA_FEMALE = Beta('BETA_FEMALE',0,-10,10,0)
BETA_FIXED_ARRIVAL_TIME = Beta('BETA_FIXED_ARRIVAL_TIME',0,-10,10,0)

rail_time  = DefineVariable('rail_time',rail_ivtt + rail_acc_time + rail_egr_time, database)
car_time = DefineVariable('car_time',car_ivtt + car_walk_time, database)

rate_G2E  = DefineVariable('rate_G2E',0.44378022,database) #from Guilders to euros
car_cost_euro  = DefineVariable('car_cost_euro',car_cost * rate_G2E,database)
rail_cost_euro  = DefineVariable('rail_cost_euro', rail_cost * rate_G2E,database)

age1 = DefineVariable('age1',(age == 0), database) #40 years or younger
age2 = DefineVariable('age2',(age == 1), database) #41 years or older
employed = DefineVariable('employed',(employ_status == 0), database)

# Utilities
V_CAR = ASC_CAR + BETA_TIME_CAR * car_time + BETA_COST_AGE1 * car_cost_euro * age1 + BETA_COST_AGE2 * car_cost_euro  * age2
V_RAIL = BETA_TIME_RAIL * rail_time + BETA_COST_AGE1 * rail_cost_euro * age1 + BETA_COST_AGE2 * rail_cost_euro * age2 + BETA_FEMALE * gender + BETA_FIXED_ARRIVAL_TIME * arrival_time

V = {0: V_CAR, 1: V_RAIL}
av = {0: 1,1: 1}

In [ ]:
#SIMULATION
prob_car = logit(V,av,0)
prob_rail = logit(V,av,1)

simulate = {
'Weighted prob. car': sampleNormalizedWeight * prob_car,
'Weighted prob. rail': sampleNormalizedWeight * prob_rail,
'Prob. car': prob_car,
'Prob. rail': prob_rail
}

biogeme = bio.BIOGEME(database, simulate)
biogeme.modelName = "Netherlands_Base_Simul"

betas = biogeme.freeBetaNames
results = res.bioResults(pickleFile = 'Netherlands_Base_Model.pickle')
betaValues = results.getBetaValues()

simulatedValues = biogeme.simulate(betaValues)

marketShare_car = 100 * simulatedValues['Weighted prob. car'].mean()
marketShare_rail = 100 * simulatedValues['Weighted prob. rail'].mean()

print('Market share car: {} %'.format(marketShare_car))
print('Market share rail: {} %'.format(marketShare_rail))

simulatedValues['simulated choice'] = simulatedValues.apply(lambda row: int(row['Prob. rail'] > 0.5), axis = 1) 
diff = simulatedValues['simulated choice'] - pandas.choice

sim_car_real_rail = 100*len(diff[diff < 0].index)/S
sim_rail_real_car = 100*len(diff[diff > 0].index)/S

print('Share of users choosing car with a higher probability for rail: {} %'.format(sim_rail_real_car))
print('Share of users choosing rail with a higher probability for car: {} %'.format(sim_car_real_rail))